# New Features in v0.1.0

This notebook demonstrates the new features added in the v0.1.0 release:
- New Screeners (Bond, Futures, Coin)
- Fluent API (select, where, select_all)
- Index Filtering (set_index)
- Field Discovery (search, technicals)
- Field Presets
- Time Interval Support
- Specific Symbol Queries

In [ ]:
%load_ext autoreload
%autoreload 2

## New Screeners

In addition to Stock, Forex, and Crypto screeners, we now have Bond, Futures, and Coin screeners.

In [ ]:
from tvscreener import BondScreener, BondField

bs = BondScreener()
df = bs.get()
print(f"Bond screener returned {len(df)} rows with {len(df.columns)} columns")
df.head()

In [ ]:
from tvscreener import FuturesScreener, FuturesField

fs = FuturesScreener()
df = fs.get()
print(f"Futures screener returned {len(df)} rows with {len(df.columns)} columns")
df.head()

In [ ]:
from tvscreener import CoinScreener, CoinField

cs = CoinScreener()
df = cs.get()
print(f"Coin screener returned {len(df)} rows with {len(df.columns)} columns")
df.head()

## Fluent API

Use `select()` and `where()` for cleaner, more readable code.

### New Pythonic Comparison Syntax (v0.1.0+)

You can now use Python comparison operators directly on fields:

In [ ]:
from tvscreener import StockScreener, StockField, FilterOperator

ss = StockScreener()

# New Pythonic syntax - use comparison operators directly!
ss.where(StockField.PRICE > 100)
ss.where(StockField.VOLUME >= 1_000_000)
ss.where(StockField.MARKET_CAPITALIZATION.between(1e9, 100e9))

# Select specific fields
ss.select(
    StockField.NAME,
    StockField.PRICE,
    StockField.CHANGE_PERCENT,
    StockField.VOLUME,
    StockField.MARKET_CAPITALIZATION
)

df = ss.get()
print(f"Found {len(df)} stocks matching criteria")
df.head(10)

### Available Comparison Operators

| Operator | Example | Description |
|----------|---------|-------------|
| `>` | `StockField.PRICE > 100` | Greater than |
| `>=` | `StockField.VOLUME >= 1e6` | Greater than or equal |
| `<` | `StockField.RSI < 30` | Less than |
| `<=` | `StockField.PRICE <= 50` | Less than or equal |
| `==` | `StockField.SECTOR == 'Tech'` | Equal to |
| `!=` | `StockField.SECTOR != 'Finance'` | Not equal to |
| `.between()` | `StockField.PRICE.between(50, 100)` | In range (inclusive) |
| `.not_between()` | `StockField.PRICE.not_between(50, 100)` | Outside range |
| `.isin()` | `StockField.SECTOR.isin(['Tech', 'Health'])` | In list |
| `.not_in()` | `StockField.SECTOR.not_in(['Finance'])` | Not in list |

In [ ]:
# More comparison examples
ss = StockScreener()

# RSI oversold stocks
ss.where(StockField.RELATIVE_STRENGTH_INDEX_14 < 30)

# With time interval (1-hour RSI)
rsi_1h = StockField.RELATIVE_STRENGTH_INDEX_14.with_interval('60')
ss.where(rsi_1h < 30)

# Filter by sector
ss.where(StockField.SECTOR.isin(['Electronic Technology', 'Health Technology']))

ss.set_range(0, 20)
df = ss.get()
print(f"Found {len(df)} oversold tech/health stocks")
df.head()

## Index Filtering

Filter results to only include stocks from specific indices (S&P 500, NASDAQ 100, Russell 2000, etc.).

In [ ]:
from tvscreener import StockScreener, IndexSymbol

# Filter to S&P 500 constituents
ss = StockScreener()
ss.set_index(IndexSymbol.SP500)
ss.set_range(0, 500)

df = ss.get()
print(f"Found {len(df)} S&P 500 stocks")
df.head(10)

In [ ]:
# Combine multiple indices
ss = StockScreener()
ss.set_index(IndexSymbol.NASDAQ_100, IndexSymbol.DOW_JONES)

df = ss.get()
print(f"Found {len(df)} stocks from NASDAQ 100 + Dow Jones")
df.head()

In [ ]:
# Search available indices
print("Available major indices:")
for idx in [IndexSymbol.SP500, IndexSymbol.NASDAQ_100, IndexSymbol.DOW_JONES, 
            IndexSymbol.RUSSELL_2000, IndexSymbol.RUSSELL_1000]:
    print(f"  {idx.name}: {idx.label}")

print("\nSearch for sector indices:")
for idx in IndexSymbol.search("technology"):
    print(f"  {idx.name}: {idx.label}")

## Select All Fields

Use `select_all()` to fetch all ~3500 available fields for a screener type.

In [ ]:
from tvscreener import StockScreener

ss = StockScreener()
ss.select_all()  # Request all ~3500 stock fields
ss.set_range(0, 10)  # Limit to 10 stocks for demo

df = ss.get()
print(f"Fetched {len(df.columns)} fields for {len(df)} stocks")
print(f"\nFirst 20 columns: {list(df.columns[:20])}")
print(f"Last 20 columns: {list(df.columns[-20:])}")

## Specific Symbol Queries

Fetch data for a specific list of symbols using the `symbols` attribute.

In [ ]:
from tvscreener import StockScreener

ss = StockScreener()

# Set specific tickers (use EXCHANGE:SYMBOL format)
ss.symbols = {
    "query": {"types": []},
    "tickers": ["NASDAQ:AAPL", "NASDAQ:MSFT", "NASDAQ:GOOGL", "NYSE:IBM", "NYSE:JPM"]
}

df = ss.get()
print(f"Fetched data for {len(df)} specific symbols")
df

In [ ]:
# Combine specific symbols with select_all() to get ALL ~3500 fields
ss = StockScreener()
ss.symbols = {
    "query": {"types": []},
    "tickers": ["NASDAQ:AAPL", "NASDAQ:NVDA", "NASDAQ:TSLA"]
}
ss.select_all()

df = ss.get()
print(f"Fetched {len(df.columns)} fields for {len(df)} symbols")
df

## Field Discovery

With 13,000+ fields, you need tools to find what you're looking for.

In [ ]:
from tvscreener import StockField, CryptoField, ForexField, BondField, FuturesField, CoinField

print("Field counts by screener type:")
print(f"  StockField:   {len(list(StockField))} fields")
print(f"  CryptoField:  {len(list(CryptoField))} fields")
print(f"  ForexField:   {len(list(ForexField))} fields")
print(f"  BondField:    {len(list(BondField))} fields")
print(f"  FuturesField: {len(list(FuturesField))} fields")
print(f"  CoinField:    {len(list(CoinField))} fields")

In [ ]:
# Search for fields by name or label
rsi_fields = StockField.search("rsi")
print(f"Found {len(rsi_fields)} RSI-related fields:")
for f in rsi_fields[:10]:
    print(f"  {f.name}: {f.label}")
print(f"  ... and {len(rsi_fields) - 10} more")

In [ ]:
# Search for MACD fields
macd_fields = StockField.search("macd")
print(f"Found {len(macd_fields)} MACD-related fields:")
for f in macd_fields[:10]:
    print(f"  {f.name}: {f.label}")

In [ ]:
# Get all technical indicator fields (those supporting time intervals)
technicals = StockField.technicals()
print(f"Found {len(technicals)} technical indicator fields")
print("\nExamples:")
for f in technicals[:5]:
    print(f"  {f.name}: {f.label}")

In [ ]:
# Get recommendation fields
recos = StockField.recommendations()
print(f"Found {len(recos)} recommendation fields:")
for f in recos[:10]:
    print(f"  {f.name}: {f.label}")

## Field Presets

Use curated field groups for common analysis needs.

In [ ]:
from tvscreener import list_presets, get_preset

# List all available presets
presets = list_presets()
print(f"Available presets ({len(presets)}):")
for p in presets:
    print(f"  {p}")

In [ ]:
from tvscreener import StockScreener, STOCK_VALUATION_FIELDS, STOCK_DIVIDEND_FIELDS

ss = StockScreener()

# Combine presets for value/dividend analysis
ss.specific_fields = STOCK_VALUATION_FIELDS + STOCK_DIVIDEND_FIELDS

df = ss.get()
print(f"Valuation + Dividend analysis with {len(df.columns)} columns")
df.head()

In [ ]:
# Get preset by name
perf_fields = get_preset('stock_performance')
print(f"stock_performance preset has {len(perf_fields)} fields:")
for f in perf_fields:
    print(f"  {f.label}")

In [ ]:
from tvscreener import CryptoScreener, CRYPTO_TECHNICAL_FIELDS

cs = CryptoScreener()
cs.specific_fields = CRYPTO_TECHNICAL_FIELDS

df = cs.get()
print(f"Crypto technical analysis with {len(df.columns)} columns")
df.head()

## Time Intervals for Technical Fields

Technical indicators can be requested with different time intervals.

In [ ]:
from tvscreener import StockScreener, StockField

ss = StockScreener()

# Get RSI on different timeframes
rsi_1h = StockField.RELATIVE_STRENGTH_INDEX_14.with_interval("60")   # 1-hour
rsi_4h = StockField.RELATIVE_STRENGTH_INDEX_14.with_interval("240")  # 4-hour
rsi_1d = StockField.RELATIVE_STRENGTH_INDEX_14.with_interval("1D")   # Daily

print(f"RSI 1h field_name: {rsi_1h.field_name}")
print(f"RSI 4h field_name: {rsi_4h.field_name}")
print(f"RSI 1D field_name: {rsi_1d.field_name}")

In [ ]:
from tvscreener import StockScreener, StockField

ss = StockScreener()

# Multi-timeframe RSI analysis
ss.specific_fields = [
    StockField.NAME,
    StockField.PRICE,
    StockField.RELATIVE_STRENGTH_INDEX_14,  # Default timeframe
    StockField.RELATIVE_STRENGTH_INDEX_14.with_interval("60"),   # 1-hour
    StockField.RELATIVE_STRENGTH_INDEX_14.with_interval("240"),  # 4-hour
]

df = ss.get()
df.head(10)

## Type-Safe Validation

The library now validates that you're using the correct field types with each screener.

In [ ]:
from tvscreener import StockScreener, ForexField, FilterOperator

ss = StockScreener()

# This will raise a TypeError - using ForexField with StockScreener
try:
    ss.add_filter(ForexField.NAME, FilterOperator.EQUAL, "test")
except TypeError as e:
    print(f"TypeError caught: {e}")

## Summary

The v0.1.0 release adds:

| Feature | Description |
|---------|-------------|
| New Screeners | BondScreener, FuturesScreener, CoinScreener |
| **Pythonic Comparisons** | `StockField.PRICE > 100`, `.between()`, `.isin()` |
| Fluent API | `select()` and `where()` methods for cleaner code |
| Index Filtering | `set_index()` to filter by index constituents (S&P 500, NASDAQ 100, etc.) |
| Select All Fields | `select_all()` to fetch all ~3500 available fields |
| Specific Symbols | Set `symbols` attribute to query specific tickers |
| Field Discovery | `search()`, `technicals()`, `recommendations()` |
| Field Presets | 23 curated field groups for common use cases |
| Time Intervals | `with_interval()` for technical indicators |
| Type Validation | Catches field/screener mismatches early |